<a href="https://colab.research.google.com/github/Jaime44/WorkSpace/blob/main/models/content%20filter%20colaborative/content-based-recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/mehmetisik/content-based-recommendation

In [1]:
path_models_colab = 'Models/content filter colaborative'

In [2]:
import os
# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/'+path_models_colab

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = os.getcwd().replace("\\", "/")

datasets_path = "/datasets/"
path_absolute = path_absolute+datasets_path

El código se está ejecutando en Google Colab.
Mounted at /content/drive
Contenido de la carpeta en Google Drive:
['datasets', 'CF_The_age_of_recommeder_System_Original.ipynb', 'Content-Based_and_Tensorflow_Recommender_System_Original.ipynb', 'CF_The_age_of_recommeder_System.ipynb', 'Content-Based_and_Tensorflow_Recommender_System.ipynb', 'content-based-recommendation_Original.ipynb', 'content-based-recommendation.ipynb', 'movie-recommendation-system.ipynb']


![CBR](https://miro.medium.com/v2/resize:fit:1400/1*H_MMnrpLQrqTSJHdDOCMoA.png)

# What is Content Based Recommendation

Content-based recommendation, also known as content-based filtering, is a type of system or algorithm that provides recommendations to a user based on their interests and preferences. Those with such recommendation systems analyze the user's past preferences and likes, and suggest new items based on similar content.

Content-based recommendation analyzes the content of items and determines the ones that are suitable for the user based on similarity criteria. For example, when making a movie recommendation, the system can take into account the genres, actors, directors, and other features of the movies the user has liked or watched. Based on this information, the system suggests other movies with similar characteristics.

This recommendation system can utilize text analysis, tagging, categorization, or other content features along with the user profile or history to better understand the user's preferences. For instance, when making a music recommendation, the system can analyze features such as genres, instruments, tempo, and rhythm.

Content-based recommendation systems can be effective in providing personalized recommendations based on user preferences. The recommended items based on the user's past data can capture their interest and provide a better user experience.

# Business Problem
To recommend movies similar to the movies that a person who comes to our site to watch movies.

# Road Map

- 1. Creating the **TF-IDF Matrix**
- 2. Creation of **Cosine Similarity Matrix**
- 3. Making Recommendations Based on Similarities
- 4. Preparation of the Study Script


In [3]:
# import Required Libraries

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [5]:
# Loading the Data Set
# df = pd.read_csv(path_absolute+"movies_metadata.csv")


In [6]:
df = pd.read_csv(path_absolute+"df_mezclado_tags_ratings_movies_links_genTags.csv")

In [7]:
tamano_muestra = 35000
df = df.sample(n=tamano_muestra, random_state=42)

In [8]:
df.head()

,userId,movieId,rating,timestamp_valoraciones,tag_df_mezclado_tags_ratings_movies_links_genMov,timestamp_etiquetas,title,genres,imdbId,tmdbId,tagId,relevance,tag_etiquetas_genómicas
1561133,60101,2571,5.0,1053445162,dystopia,1244667471,"Matrix, The (1999)",Action|Sci-Fi|Thriller,133093,603.0,512,0.12625,hitman
1499330,42923,1228,4.0,1330057917,classic,1330057913,Raging Bull (1980),Drama,81398,1578.0,710,0.60700,nocturnal
1048680,7620,34532,5.0,1144937579,Magick,1144937622,"Skeleton Key, The (2005)",Drama|Horror|Mystery|Thriller,397101,9913.0,1047,0.05750,true story
1703441,62199,5636,3.0,1199318408,Anthony Russo,1328741490,Welcome to Collinwood (2002),Comedy|Crime,271259,9260.0,27,0.05325,adoption
5648649,29801,6250,2.0,1476349149,Stephen King,1476349186,Dreamcatcher (2003),Drama|Horror|Sci-Fi|Thriller,285531,6171.0,276,0.36500,cult film


In [9]:
# Eliminar las columnas imdbId y tmdbId
# Renombrar las columnas tal que: 'timestamp_valoraciones': 'timestamp_scr' y 'timestamp_etiquetas': 'timestamp_tags'
#Mover las columnas de rating y relevance al final y la de tag_etiquetas_genómicas despues de tag_df_mezclado...

# Inserta la columna 'rating' en la última posición del DataFrame
df.insert(len(df.columns)-1, 'rating', df.pop('rating'))
df.insert(len(df.columns)-2, 'relevance', df.pop('relevance'))
df.insert(5, 'tag_etiquetas_genómicas', df.pop('tag_etiquetas_genómicas'))

df.rename(columns={'timestamp_valoraciones': 'timestamp_rt', 'timestamp_etiquetas': 'timestamp_tags', 'tag_df_mezclado_tags_ratings_movies_links_genMov': 'tag_by_user', 'tag_etiquetas_genómicas': 'gen_tag'}, inplace=True)

columnas_a_eliminar = ['imdbId', 'tmdbId']
# Elimina las columnas especificadas del DataFrame 'data'
df = df.drop(columnas_a_eliminar, axis=1)

In [10]:
df.head()

,userId,movieId,timestamp_rt,tag_by_user,timestamp_tags,gen_tag,title,genres,tagId,relevance,rating
1561133,60101,2571,1053445162,dystopia,1244667471,hitman,"Matrix, The (1999)",Action|Sci-Fi|Thriller,512,0.12625,5.0
1499330,42923,1228,1330057917,classic,1330057913,nocturnal,Raging Bull (1980),Drama,710,0.60700,4.0
1048680,7620,34532,1144937579,Magick,1144937622,true story,"Skeleton Key, The (2005)",Drama|Horror|Mystery|Thriller,1047,0.05750,5.0
1703441,62199,5636,1199318408,Anthony Russo,1328741490,adoption,Welcome to Collinwood (2002),Comedy|Crime,27,0.05325,3.0
5648649,29801,6250,1476349149,Stephen King,1476349186,cult film,Dreamcatcher (2003),Drama|Horror|Sci-Fi|Thriller,276,0.36500,2.0


In [11]:
df.shape

(35000, 11)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35000 entries, 1561133 to 549885
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   userId          35000 non-null  int64  
 1   movieId         35000 non-null  int64  
 2   timestamp_rt    35000 non-null  int64  
 3   tag_by_user     35000 non-null  object 
 4   timestamp_tags  35000 non-null  int64  
 5   gen_tag         35000 non-null  object 
 6   title           35000 non-null  object 
 7   genres          35000 non-null  object 
 8   tagId           35000 non-null  int64  
 9   relevance       35000 non-null  float64
 10  rating          35000 non-null  float64
dtypes: float64(2), int64(5), object(4)
memory usage: 3.2+ MB


In [13]:
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = df.isnull().sum()

# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = df.isnull().sum().sum()

# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)

# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

Valores nulos por columna:
userId            0
movieId           0
timestamp_rt      0
tag_by_user       0
timestamp_tags    0
gen_tag           0
title             0
genres            0
tagId             0
relevance         0
rating            0
dtype: int64

Total de valores nulos en el dataset: 0


In [14]:
# Seleccionar las filas con valores nulos en la columna deseada
filas_con_nulos = df[df['tag_by_user'].isnull()]

# Mostrar las filas con valores nulos
print("Filas con valores nulos en la columna tag:")
print(filas_con_nulos)

# Eliminar las filas con valores nulos en la columna deseada
df = df.dropna(subset=['tag_by_user'])
print("DataFrame después de eliminar filas con valores nulos:")
# Cuenta los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = df.isnull().sum()
# Cuenta los valores nulos en todo el DataFrame
total_valores_nulos = df.isnull().sum().sum()
# Imprime la cantidad de valores nulos por columna
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
# Imprime el total de valores nulos en el DataFrame
print("\nTotal de valores nulos en el dataset:", total_valores_nulos)

Filas con valores nulos en la columna tag:
Empty DataFrame
Columns: [userId, movieId, timestamp_rt, tag_by_user, timestamp_tags, gen_tag, title, genres, tagId, relevance, rating]
Index: []
DataFrame después de eliminar filas con valores nulos:
Valores nulos por columna:
userId            0
movieId           0
timestamp_rt      0
tag_by_user       0
timestamp_tags    0
gen_tag           0
title             0
genres            0
tagId             0
relevance         0
rating            0
dtype: int64

Total de valores nulos en el dataset: 0


In [15]:
df.shape

(35000, 11)

In [16]:
content_df = df[['title', 'genres', 'tag_by_user', 'gen_tag', 'relevance', 'rating']]

In [17]:
content_df = content_df.dropna(subset=['tag_by_user'])

In [18]:
import re
import string

def remove_bars(text):
    # Eliminar barras y dejar palabras separadas por un espacio
    cleaned = re.sub(r'\|', ' ', text)
    return cleaned

def separate(text):
    clean_text = []
    for t in text.split(','):
        cleaned = re.sub('\(.*\)', '', t) # Remove text inside parentheses
        cleaned = cleaned.translate(str.maketrans('','', string.digits))
        cleaned = cleaned.replace(' ', '')
        cleaned = cleaned.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text.append(cleaned)
    return ' '.join(clean_text)

def remove_punc(text):
    try:
        cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text = cleaned.translate(str.maketrans('','', string.digits))
    except Exception as e:
        print(f"ERROR -----------------------------------------> {e} AND {text}")
    return clean_text

In [19]:
# Ejemplo de uso: Pixar Animation Studios Warner Bros., Lancaster Gate
text_with_bars = "oldie but goodie"

# text_with_bars = "Pixar Animation Studios"

# text_with_bars = "Warner Bros., Lancaster Gate"

cleaned_text = remove_punc(text_with_bars)
print(cleaned_text)

oldie but goodie


In [20]:
content_df['genres'] = (content_df['genres'].apply(remove_bars)).apply(remove_punc)
content_df['tag_by_user'] = content_df['tag_by_user'].apply(remove_punc)
# content_df.set_index('title', inplace=True)
content_df = content_df.reset_index()

In [21]:
content_df.head()

,index,title,genres,tag_by_user,gen_tag,relevance,rating
0,1561133,"Matrix, The (1999)",action scifi thriller,dystopia,hitman,0.12625,5.0
1,1499330,Raging Bull (1980),drama,classic,nocturnal,0.60700,4.0
2,1048680,"Skeleton Key, The (2005)",drama horror mystery thriller,magick,true story,0.05750,5.0
3,1703441,Welcome to Collinwood (2002),comedy crime,anthony russo,adoption,0.05325,3.0
4,5648649,Dreamcatcher (2003),drama horror scifi thriller,stephen king,cult film,0.36500,2.0


In [22]:
content_df_aux = content_df.copy()
content_df_aux = content_df_aux[['genres', 'tag_by_user', 'gen_tag']]
content_df_aux['bag_of_words'] = ''
content_df_aux.loc[:, 'bag_of_words'] = content_df_aux.loc[:, content_df_aux.columns[0:]].apply(lambda x: ' '.join(x), axis=1)
# content_df.set_index('original_title', inplace=True)
columnas_a_eliminar = ['tag_by_user', 'gen_tag']
# Elimina las columnas especificadas del DataFrame 'data'
content_df = content_df.drop(columnas_a_eliminar, axis=1)

content_df['genres'] = content_df_aux['bag_of_words']
content_df.rename(columns={'genres': 'bag_of_words'}, inplace=True)

content_df.head()

,index,title,bag_of_words,relevance,rating
0,1561133,"Matrix, The (1999)",action scifi thriller dystopia hitman,0.12625,5.0
1,1499330,Raging Bull (1980),drama classic nocturnal,0.60700,4.0
2,1048680,"Skeleton Key, The (2005)",drama horror mystery thriller magick true story,0.05750,5.0
3,1703441,Welcome to Collinwood (2002),comedy crime anthony russo adoption,0.05325,3.0
4,5648649,Dreamcatcher (2003),drama horror scifi thriller stephen king cult ...,0.36500,2.0


# 1. Creating the TF-IDF Matrix

In [23]:
df = content_df.copy()

In [24]:
df["bag_of_words"].head()

0               action scifi thriller dystopia hitman 
1                             drama classic nocturnal 
2     drama horror mystery thriller magick true story 
3                 comedy crime anthony russo adoption 
4    drama horror scifi thriller stephen king cult ...
Name: bag_of_words, dtype: object

In [25]:
df["bag_of_words"].isnull().sum()

0

In [26]:
# Convertir la colección de textos (en este caso, la bolsa de palabras)
# en una matriz TF-IDF.

# Vamos a eliminar las construcciones como a, an, the, and,
# pero que no tienen sentido para nosotros de nuestros DataFrames.
tfidf = TfidfVectorizer(stop_words="english")

In [27]:
# Rellenemos el valor nulo de la variable cin bag_of_words con nada para evitar errores en los siguientes pasos
df['bag_of_words'] = df['bag_of_words'].fillna('')

In [28]:
df["bag_of_words"].isnull().sum()

0

In [29]:
# ajustar y transformar según el objeto tfidf
# Los de las filas son textos 'bolsa de palabras'. Los de las columnas son palabras únicas.

# Crear la matriz TF-IDF (Term Frequency-Inverse Document Frequency) a partir de la columna
# 'bag_of_words' del conjunto de datos. La matriz TF-IDF es una representación numérica
# de los documentos en función de la frecuencia de las palabras que contienen y de
# su importancia en el conjunto de documentos.
tfidf_matrix = tfidf.fit_transform(df['bag_of_words'])

In [30]:
# Cada fila representa un documento (en este caso, una película) y
# cada columna representa una palabra única en el conjunto de datos.

# Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada documento.
tfidf_matrix.shape

(35000, 7258)

In [31]:
#Si queremos ver todas las palabras únicas de las columnas

# Filas: Cada fila de la matriz corresponde a una película en el conjunto de datos.
# Por ejemplo, si tienes 10,000 películas, habrá 10,000 filas en la matriz.

# Columnas: Cada columna representa una palabra única en el conjunto de datos.
# La cantidad de columnas es igual al número total de palabras únicas en todas
# las bolsas de palabras de películas.

# Valores: Los valores en la matriz son las puntuaciones TF-IDF para cada palabra en cada película.
# Estas puntuaciones miden la importancia relativa de una palabra en un documento en comparación
# con su frecuencia en el conjunto de documentos. Valores más altos indican que la palabra
# es más importante en el contexto de ese documento.

tfidf.get_feature_names_out()


array(['007', '100', '11', ..., 'zucker', 'zydeco', '奇趣马戏团'], dtype=object)

In [32]:
# tfidf scores
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 2. Creation of Cosine Similarity Matrix

In [33]:
# calcula la similitud coseno entre todas las filas de la matriz TF-IDF.
# La similitud coseno mide el coseno del ángulo entre dos vectores y
# proporciona una medida de cuán similares son dos documentos.

# Calcula cos sim para todos los pares de documentos posibles uno por uno. En la matriz cosine_sim,
# cada película tiene similitudes entre sí

# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [34]:
import cupy as cp

# Convertir la matriz TF-IDF a cupy array
tfidf_matrix_cupy = cp.array(tfidf_matrix.toarray())

# Calcular la similitud del coseno utilizando cupy.matmul
cosine_sim_cupy = cp.matmul(tfidf_matrix_cupy, tfidf_matrix_cupy.T)

cosine_sim = cosine_sim_cupy


In [35]:
cosine_sim = cosine_sim_cupy

In [36]:
# cosine_sim[i, j] representa la similitud coseno entre la película con el índice i y
# la película con el índice j en el conjunto de datos.

cosine_sim.shape

(35000, 35000)

In [ ]:
cosine_sim

In [ ]:
# Para ver en qué se parece la película del índice 1 a todas las demás películas

cosine_sim[1]

In [ ]:
cosine_sim[1].shape

# 3. Making Recommendations Based on Similarities

In [ ]:
# Vamos a crear una serie pd de índices y nombres de películas
# se crea una Serie de Pandas llamada indices donde los índices son
# los títulos de las películas y los valores son los índices originales del DataFrame df.
indices = pd.Series(df.index, index=df['title'])

In [ ]:
indices.head()

In [ ]:
indices.shape

In [ ]:
# contemos la información del índice de las películas y simplifiquemos las películas más repetitivas a las más recientes

indices.index.value_counts().head()

In [ ]:
# Se eliminan las películas duplicadas del índice, conservando solo la última aparición de cada película.
# Esto se hace para evitar ambigüedades y asegurar que cada película tenga un índice único.
indices = indices[~indices.index.duplicated(keep='last')]

In [ ]:
indices.index.value_counts().head()

In [ ]:
indices.shape

In [ ]:
indices["Matrix, The (1999)"]

In [ ]:
indices["Wolf of Wall Street, The (2013)"]

In [ ]:
# Asigno el índice de la película "Sherlock Holmes" a la variable
movie_title = "Wolf of Wall Street, The (2013)"
movie_index = indices[movie_title]

In [ ]:
cosine_sim[movie_index]

In [ ]:
cosine_sim[movie_index].shape

In [ ]:
# Veamos los Smilarity Scores que expresan las similitudes entre la película "Wolf of Wall Street, The (2013)" y otras películas
# similarity_scores = pd.DataFrame(cosine_sim[movie_index],
#                                  columns=["score"])

similarity_scores = pd.DataFrame(cosine_sim[movie_index].get(), columns=["score"])

In [ ]:
# Las similitudes entre la película 'Sherlock Holmes' y el resto de películas

similarity_scores.head()

In [ ]:
similarity_scores.shape

In [ ]:
movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index

In [ ]:
import pandas as pd

# Supongamos que tu DataFrame se llama df y la Serie de índices es index_series
# index_series = Int64Index([9859, 3228, 1656, 8951, 6811, 998, 5030, 6972, 8591, 4820, ...])

# Paso 1
duplicated_titles = df.loc[movie_indices_sort, 'title'].duplicated(keep='first')

# Paso 2
indices_sin_duplicados = movie_indices_sort[~duplicated_titles]

# Paso 3
filas_sin_duplicados = df.loc[indices_sin_duplicados]

# Ahora filas_sin_duplicados contiene las filas correspondientes a los índices
# en index_series, eliminando las filas con títulos duplicados, manteniendo la primera ocurrencia.

# Supongamos que el string que deseas excluir es 'The Promise'
string_a_excluir = movie_title

# Filtrar las filas que no contienen el string a excluir en la columna 'title'
filas_sin_duplicados_y_exclusion = df.loc[indices_sin_duplicados]
filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != string_a_excluir]

In [ ]:
# Supongamos que el string que deseas excluir es 'The Promise'
string_a_excluir = 'Wolf of Wall Street, The (2013)'

# Filtrar las filas que no contienen el string a excluir en la columna 'title'
filas_sin_duplicados_y_exclusion = df.loc[indices_sin_duplicados]
filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != string_a_excluir]


In [ ]:
filas_sin_duplicados_y_exclusion.head()

In [ ]:
# Vamos a listar las puntuaciones de similitud de la película 'Sherlock Holmes' en orden descendente. Empieza por 1 porque es la primera película.

# movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index

In [ ]:
# Go to the indexes we selected in our first data set

# df['title'].iloc[movie_indices]

# 4. Preparation of the Study Script

In [ ]:
# def content_based_recommender(title, cosine_sim, dataframe):
#     # create indexes
#     indices = pd.Series(dataframe.index, index=dataframe['title'])
#     indices = indices[~indices.index.duplicated(keep='last')]
#     # capturing index of title
#     movie_index = indices[title]
#     # calculate similarity scores based on title
#     similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
#     # Bringing the top 10 movies except for itself
#     movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
#     return dataframe['title'].iloc[movie_indices]

In [ ]:
def content_based_recommender(title, cosine_sim, dataframe):
    # create indexes
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    # capturing index of title
    movie_index = indices[title]
    # calculate similarity scores based on title
    # similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])

    similarity_scores = pd.DataFrame(cosine_sim[movie_index].get(), columns=["score"])

    # Bringing the top 10 movies except for itself
    movie_indices_sort = similarity_scores.sort_values("score", ascending=False).index
    duplicated_titles = dataframe.loc[movie_indices_sort, 'title'].duplicated(keep='first')
    indices_sin_duplicados = movie_indices_sort[~duplicated_titles]
    filas_sin_duplicados_y_exclusion = dataframe.loc[indices_sin_duplicados]
    filas_sin_duplicados_y_exclusion = filas_sin_duplicados_y_exclusion[filas_sin_duplicados_y_exclusion['title'] != title]
    return filas_sin_duplicados_y_exclusion['title'].head(10)

In [ ]:
content_based_recommender("Wolf of Wall Street, The (2013)", cosine_sim, df)

In [ ]:
content_based_recommender("Matrix, The (1999)", cosine_sim, df)

In [ ]:
content_based_recommender("Star Wars: Episode IV - A New Hope (1977)", cosine_sim, df)

In [ ]:
content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)

In [ ]:
def calculate_cosine_sim(dataframe):
    tfidf = TfidfVectorizer(stop_words='english')
    dataframe['bag_of_words'] = dataframe['bag_of_words'].fillna('')
    tfidf_matrix = tfidf.fit_transform(dataframe['bag_of_words'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [ ]:
# cosine_sim = calculate_cosine_sim(df)
# content_based_recommender('Pulp Fiction (1994)', cosine_sim, df)